In [1]:
%matplotlib inline

In [2]:
import json
import os
from copy import copy
from functools import partial
from os.path import basename, join
from subprocess import call
from urllib.request import urlopen

import pyproj
import requests

import geopandas as gpd
import pandas as pd
import us
from shapely.geometry import LineString, Point, shape
from shapely.ops import transform

In [3]:
def gdf_from_geojson(geojson, crs='epsg:4326'):
    """Convert a GeoJSON dict to GeoDataFrame"""
    def _f(f):
        f['properties']['geometry'] = shape(f['geometry'])
        return f['properties']
    
    return gpd.GeoDataFrame([_f(f) for f in geojson['features']], crs=crs)

In [4]:
fips_to_abbr = {st.fips: st.abbr for st in us.states.STATES_AND_TERRITORIES if st.fips}

states geodataframe

In [5]:
states = gpd.read_file('https://gist.githubusercontent.com/simonkassel/d091fc86253b65c68bb644443c74f366/raw/001b6ad8232e2ecfdc5fbd46a5ef8f2a9642e94d/us_states.geojson')
states.rename(columns={'NAME': 'ST_NAME', 'abbr': 'ST_ABBR', 'STATEFP':'STATE_FP'}, inplace=True)
states['ST_NAME'] = states['ST_NAME'].apply(lambda x: x.upper())

counties geodataframe

In [6]:
with urlopen("https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_20m.json") as url:
    data = json.loads(url.read().decode("ISO-8859-1"))
counties = gdf_from_geojson(data)
counties.rename(columns={'STATE':'STATE_FP', 'COUNTY': 'COUNTY_FP', 'NAME': 'COUNTY_NAME'}, inplace=True)
counties['COUNTY_NAME'] = counties['COUNTY_NAME'].apply(lambda x: x.upper())
counties['ST_ABBR'] = counties['STATE_FP'].apply(lambda x: fips_to_abbr[x])

hcris dataframe

In [7]:
if not os.path.isdir('download_data'):
    os.mkdir('download_data')

url = 'http://downloads.cms.gov/files/hcris/hosp10-reports.zip'
z = basename(url)
c = 'download_data/HOSPITAL10_PROVIDER_ID_INFO.CSV'

if not os.path.exists(c):
    call('wget -O download_data/{} {}'.format(z, url), shell=True)
    call('cd download_data && unzip -o {}'.format(z), shell=True)
    
hcris = pd.read_csv(c)

hcris.rename(columns = {'City': 'CITY_NAME', 'State': 'ST_ABBR', 'County': 'COUNTY_NAME'}, inplace=True)

In [8]:
# provider num should be 6 char so need to zfill
hcris['PROVIDER_NUMBER'] = hcris['PROVIDER_NUMBER'].apply(lambda x: str(x).zfill(6))

# Rename this column to match up with reports
hcris = hcris.rename(columns={'PROVIDER_NUMBER': 'Provider Number'})

geocoding

In [9]:
# Input google and Mapbox api keys or retrieve from environment variable
google_key = ''
mapbox_key = ''

google_key = os.getenv('GOOGLE_API_KEY', google_key)
mapbox_key = os.getenv('MAPBOX_API_KEY', mapbox_key)

In [16]:
def google_geocode_str(name, addr, city, state, county, zip_code, key):
    base = 'https://maps.googleapis.com/maps/api/geocode/json?'
    address_str = 'address={}, {}'.format(name, addr)
    components_str = f'&components=country:US|locality:{city}|administrative_area:{state}|administrative_area:{county} county|postal_code:{zip_code}'
    key_str = '&key={}'.format(key)
    
    return base + address_str + components_str + key_str

In [17]:
def mapbox_geocode_str(name, addr, city, state, county, zip_code, key):
    base = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'
    address_str = f'{name}, {addr}, {city}, {state}, {county} county,{zip_code}.json?'
    components_str = 'country=US&limit=1'
    key_str = '&access_token={}'.format(key)
    
    return base + address_str + components_str + key_str

In [18]:
test = False
if test:
    hcris = hcris.head()

In [ ]:
results = []
for k, r in hcris.iterrows():
    if not isinstance(r['COUNTY_NAME'], str):
        admin_geom = states[states['ST_ABBR'] == r['ST_ABBR']]['geometry'].values[0]
    
    else:
        county = counties[(counties['ST_ABBR'] == r['ST_ABBR']) & (counties['COUNTY_NAME'] == r['COUNTY_NAME'])]
        if len(county) == 1:
            admin_geom = county['geometry'].values[0]
        
        elif len(county) < 1:
            rcn = r['COUNTY_NAME'].replace(' COUNTY', '')
            county = counties[(counties['ST_ABBR'] == r['ST_ABBR']) & (counties['COUNTY_NAME'] == rcn)]
            if len(county) < 1:
                admin_geom = states[states['ST_ABBR'] == r['ST_ABBR']]['geometry'].values[0]
        
        elif len(county) > 1:
            raise Exception('Multiple matching counties found')

    args = [r['HOSP10_Name'], r['Street_Addr'], r['CITY_NAME'], r['ST_ABBR'], r['COUNTY_NAME'], r['Zip_Code']]
    
    def _p(search_str):
        response = requests.get(search_str)
        if response.status_code != 200:
            print(response.json())
            raise Exception('Status code exeption: {}'.format(response.status_code))
        
        
        response = response.json()
        if 'results' in response:
            for result in response['results']:
                y, x = result['geometry']['location'].values()
                hosp_point = Point(x, y)
                if hosp_point.within(admin_geom):
                    return hosp_point
            return None
        else:
            for feature in response['features']:
                hosp_point = Point(feature['center'])
                if hosp_point.within(admin_geom):
                    return hosp_point
            return None
    
    def _dist(p1, p2):
        def _proj(p):
            project = partial(
                pyproj.transform,
                pyproj.Proj('epsg:4326'), 
                pyproj.Proj('epsg:2163'))
            return transform(project, p)
        
        return _proj(p1).distance(_proj(p2))

    # Google
    google_point = _p(google_geocode_str(*args + [google_key]).replace('#', ''))
    # Mapbox
    mapbox_point = _p(mapbox_geocode_str(*args + [mapbox_key]).replace('#', ''))
    
    # Combine
    dist = None
    geom = None
    if google_point and mapbox_point:
        dist = _dist(google_point, mapbox_point)
        geom = LineString([google_point, mapbox_point]).centroid
    elif google_point:
        geom = google_point
    elif mapbox_point:
        geom = mapbox_point
    
    results.append({'combined': geom, 'mapbox': mapbox_point, 'google': google_point, 'distance': dist})
    
    if (k + 1) % 100 == 0:
        print('Geocoded [{}] of {} hospitals.'.format(k + 1, len(hcris)))

geoms = pd.DataFrame(results)

In [ ]:
if not os.path.isdir('interim_data'):
    os.mkdir('interim_data')
    
for c in geoms.columns:
    if c != 'distance':
        hcris['geometry'] = geoms[c]
        gdf = gpd.GeoDataFrame(hcris, crs='epsg:4326')
        fname = 'interim_data/hcris_geocoded_{}.geojson'.format(c)
        if test:
            fname = fname.replace('.geojson', '-TEST.geojson')
        
        gdf.to_file(fname, driver='GeoJSON')